In [3]:
import torch
import torch.nn as nn
import data

device = torch.device("cpu")

모델 실행을 위해 torch와 torch.nn, 그리고 data.py를 import합니다.
학습은 이미 되어 있다고 가정하고, 모델을 사용하기 위해 device를 설정해 줍니다.

In [5]:
print("Load model......")

with open('model.pt', 'rb') as f:
    model = torch.load(f).to(device)
model.eval()

Load model......


RNNModel(
  (drop): Dropout(p=0.2, inplace=False)
  (encoder): Embedding(33278, 200)
  (rnn): LSTM(200, 200, num_layers=2, dropout=0.2)
  (decoder): Linear(in_features=200, out_features=33278, bias=True)
)

학습된 모델을 가져옵니다. 해당 모델은 main.py를 실행함으로서 생성됩니다.

```user@solar:~/$ python3 main.py --cuda --epochs 6```

해당 코드를 실행하면 다음과 같이 학습이 진행되는 것을 볼 수 있습니다.
![main.py 학습](ai1.PNG)
학습이 끝났다면 model.pt에 저장되게 됩니다. 이번 실습에서는 해당 모델 파일을 불러와서 문장의 확률을 계산합니다.

In [6]:
print("Load corpus......")

corpus = data.Corpus('./data/wikitext-2')
ntokens = len(corpus.dictionary)

Load corpus......


사전을 불러오기 위해 corpus를 로딩합니다. ntokens는 사전의 크기를 나타냅니다.

In [9]:
print("Compute probabilities......")

def seq_prob(seq):
    prod_probs = 1
    softmax = nn.Softmax(dim=2)
    hidden = model.init_hidden(1) # hidden값 초기화
    with torch.no_grad():
        for i in range(len(seq)-1):
            input = torch.Tensor([[corpus.dictionary.word2idx[seq[i]]]]).long().to(device) # dictionary를 이용하여 모델에 입력되는 첫 번째 input 구성
            output, hidden = model(input, hidden) # model로부터 실행 결과 얻기 
            word_weights = softmax(output) # 각 단어별 확률
            next_idx = corpus.dictionary.word2idx[seq[i+1]]
            prod_probs *= word_weights[0, 0, next_idx].double()# 단어별 확률의 곱
    return prod_probs

print('P(the dog bark.) =', seq_prob(['the', 'dog', 'bark', '.']))
print('P(the cat bark.) =', seq_prob(['the', 'cat', 'bark', '.']))
print()

Compute probabilities......
P(the dog bark.) = tensor(2.2494e-10, dtype=torch.float64)
P(the cat bark.) = tensor(3.3988e-11, dtype=torch.float64)



실습 함수입니다. seq_prob() 함수는 문장의 단어 배열을 입력으로 받아(seq) 해당 문장의 생성 확률(prod_probs)를 계산합니다.

1. 모델을 사용하기 전에, 먼저 hidden을 초기화합니다.
2. 매 단어를 모델의 input으로 넣기 위해 단어를 모델의 input으로 변환합니다.
3. 초기화한 hidden과 input을 사용하여, 모델의 output을 생성합니다.
4. output을 확률로 변환하여 word_weights를 만듭니다.
5. word_weights에서 다음 단어에 대한 확률을 찾아 pred_probs에 곱해줍니다.

해당 함수를 정확히 구현하였다면, 'the dog bark.'의 확률이 'the cat bark.'의 확률보다 큰 것을 확인하실 수 있습니다. 개는 짖지만, 고양이는 짖는다고 하지 않으니까요.

In [ ]:
# input 랜덤생성
가장큰 아웃풋 인덱스 eos가 나올때까지